<a href="https://colab.research.google.com/github/lmolinario/ML_Sec_project/blob/main/MLSEC_FNM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Machine Learning Security Project - Project 1**
###**Student: Lello Molinario (70/90/00369)**

**Instructions**

Re-evaluate 5 RobustBench models with another attack algorithm (e.g. FMN) and identify samples for which one attack works and the other doesn't. Explain the results - i.e., provide some motivations on why one of the attacks did not work properly, while the other did.


**Execution**

In order to re-evaluate 5 RobustBench models I chose "FNM" as the attack algorithm.

# **Step 1: Installing libraries and resolving incompatibilities**

In [14]:
!python --version
!sudo apt-get install python3.10 python3.10-venv python3.10-dev -y


Python 3.10.12
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10 is already the newest version (3.10.12-1~22.04.8).
python3.10-dev is already the newest version (3.10.12-1~22.04.8).
python3.10-venv is already the newest version (3.10.12-1~22.04.8).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [15]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3


There are 2 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
  0            /usr/bin/python3.11   2         auto mode
* 1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.11   2         manual mode

Press <enter> to keep the current choice[*], or type selection number: 1


In [16]:
!python3 -m pip install --upgrade pip


/usr/bin/python3: No module named pip


In [17]:
!sudo apt-get install python3.10-distutils -y
!wget https://bootstrap.pypa.io/get-pip.py -O get-pip.py
!python3.10 get-pip.py


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.10-distutils'
python3-distutils is already the newest version (3.10.8-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
--2025-01-27 19:00:09--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2275758 (2.2M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.17M  --.-KB/s    in 0.08s   

2025-01-27 19:00:09 (27.2 MB/s) - ‘get-pip.py’ saved [2275758/2275758]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.3 MB/s eta 0:00:00


In [18]:
!python3.10 -m pip --version


pip 25.0 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip3 install git+https://github.com/RobustBench/robustbench.git

  Cloning https://github.com/RobustBench/robustbench.git to /tmp/pip-req-build-gm6dlzr5
  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git /tmp/pip-req-build-gm6dlzr5
  Resolved https://github.com/RobustBench/robustbench.git to commit 46a91f44524133b2cd8f721ec7e73ecb63f17fc8
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fra31/auto-attack.git (to revision a39220048b3c9f2cca9a4d3a54604793c68eca7e) to /tmp/pip-install-uihn1kqc/autoattack_204f03d3a9a440e68fd1c8eb790e73ca
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack.git /tmp/pip-install-uihn1kqc/autoattack_204f03d3a9a440e68fd1c8eb790e73ca
  Running command git rev-parse -q --verify 'sha^a39220048b3c9f2cca9a4d3a54604793c68eca7e'
  Running command git fetch -q https://github.com/fra31/auto-attack.git a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Resolved https://github.com/fra31/auto-attack.git to commit a39220048b3c9f2cca9a4

In [ ]:
!pip install git+https://github.com/pralab/secml.git


In [2]:
!pip install foolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 42.0 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve

In [ ]:
import robustbench
import secml
import foolbox

print(f"RobustBench version: {robustbench.__name__}")
print(f"SecML version: {secml.__version__}")
print(f"Foolbox version: {foolbox.__version__}")

RobustBench version: robustbench
SecML version: 0.13.post1
Foolbox version: 3.3.4


In [ ]:
#import secml
#import foolbox as fb
#import robustbench


import os
import gc
import torch
import numpy as np

import matplotlib.pyplot as plt

from secml.array import CArray
from secml.ml import CClassifierPyTorch
from secml.ml.peval.metrics import CMetricAccuracy
#from secml.adv.attacks.evasion import CAttackEvasionFoolbox
from secml.data.loader import CDataLoaderCIFAR10
from secml.ml.features.normalization import CNormalizerMinMax
from secml.figure import CFigure
from secml.explanation import \
    CExplainerGradient, CExplainerGradientInput, CExplainerIntegratedGradients
from secml.utils import fm
from secml import settings
from secml.ml.classifiers.loss import CSoftmax

from robustbench.utils import load_model

from torch import nn

2025-01-27 15:36:58,284 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data
2025-01-27 15:36:58,284 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data


INFO:secml.settings:New `SECML_HOME_DIR` created: /root/secml-data


2025-01-27 15:36:58,288 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf
2025-01-27 15:36:58,288 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf


INFO:secml.settings:Default configuration file copied to: /root/secml-data/secml.conf


2025-01-27 15:36:58,292 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets
2025-01-27 15:36:58,292 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets


...

INFO:secml.settings:New `SECML_EXP_DIR` created: /root/secml-data/experiments


2025-01-27 15:36:58,311 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs
2025-01-27 15:36:58,311 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs


INFO:secml.settings:New `SECML_LOGS_DIR` created: /root/secml-data/logs


2025-01-27 15:36:58,317 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data
2025-01-27 15:36:58,317 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


INFO:secml.settings:New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


## Global Variables
Contains definition of global variables

In [ ]:
input_shape    = (3, 32, 32)

model_names    = [
    "Ding2020MMA",
    "Wong2020Fast",
    "Andriushchenko2020Understanding",
    "Sitawarin2020Improving",
    "Debenedetti2022Light_XCiT-L12"
]

n_samples      = 64

dataset_labels = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

## Loading models

Loads five models from robustbench. We have chosen the following models:
*   1 - Ding2020MMA
*   2 - Wong2020Fast
*   3 - Andriushchenko2020Understanding
*   4 - Sitawarin2020Improving
*   5 - Debenedetti2022Light_XCiT-L12

CIFAR-10
Linf, eps=8/255


In [ ]:
def load_model(model_name):
    """
    Load a single model from Robustbench and
    wrap it into a CClassifier
    """
    model = robustbench.utils.load_model(
        model_name=model_name,
        dataset='cifar10',
        threat_model='Linf',
    )

    clf  = CClassifierPyTorch(
        model,
        input_shape=input_shape,
        pretrained=True,
        pretrained_classes=CArray(list(range(10))),
        preprocess=None
    )

    return clf



models = []
for name in model_names:
    try:
        model = load_model(name)
        models.append(model)
    except Exception as e:
        print(f"Error loading model {name}: {e}")

  checkpoint = torch.load(model_path, map_location=torch.device('cpu'))

Downloading: "https://github.com/RobustBench/robustbench/releases/download/v1.1/debenedetti2022light-xcit-l-cifar10-linf.pth.tar" to /root/.cache/torch/hub/checkpoints/debenedetti2022light-xcit-l-cifar10-linf.pth.tar


## Loading  CIFAR-10

Loads 64 samples from CIFAR-10 dataset with shape (3, 32, 32)

In [ ]:
tr, ts      = CDataLoaderCIFAR10().load()
normalizer  = CNormalizerMinMax().fit(tr.X)

ts          = ts[:n_samples, :]

# Normalize test samples
ts.X = normalizer.transform(ts.X)

## Fast-Minimum-Norm (FMN) attack

Computes the accuracy of the models, just to confirm that it is working properly.

In [ ]:
# Calcolo delle predizioni e accuratezza dei modelli
metric = CMetricAccuracy()
models_preds = [clf.predict(ts.X) for clf in models]
accuracies = [metric.performance_score(y_true=ts.Y, y_pred=y_pred) for y_pred in models_preds]

print("-" * 90)
# Stampa delle accuratezze
for idx in range(len(model_names)):
    print(f"Model name: {model_names[idx]:<40} - Clean model accuracy: {(accuracies[idx] * 100):.2f} %")
print("-" * 90)

------------------------------------------------------------------------------------------
Model name: Ding2020MMA                              - Clean model accuracy: 84.38 %
Model name: Wong2020Fast                             - Clean model accuracy: 84.38 %
Model name: Andriushchenko2020Understanding          - Clean model accuracy: 78.12 %
Model name: Sitawarin2020Improving                   - Clean model accuracy: 82.81 %
Model name: Debenedetti2022Light_XCiT-L12            - Clean model accuracy: 90.62 %
------------------------------------------------------------------------------------------


##Runs the FMN attack on each model

In [ ]:
def FNM_attack(samples, labels, models):
    """
    Funzione per eseguire l'attacco Fast Minimum Norm (FMN) su un insieme di modelli.

    :param samples: Dati di input (es. immagini o campioni) per l'attacco.
    :param labels: Etichette corrette dei campioni.
    :param models: Lista di modelli da attaccare.
    :return: Dizionario con i dati relativi all'attacco per ciascun modello.
    """
    # Parametri iniziali per l'attacco FMN
    init_params = dict(
        steps=500,
        max_stepsize=1.0,
        gamma=0.05
    )

    attack_data = {}

    # Esegui l'attacco su ciascun modello
    for idx, model in enumerate(models):
        print(f"Inizio attacco sul modello {idx + 1}")

        try:
            # Configura l'attacco FMN
            attack = CAttackEvasionFoolbox(
                model=model,
                y_target=None,  # Attacco non mirato
                epsilons=8 / 255,  # Vincolo L∞
                fb_attack_class=fb.attacks.LInfFMNAttack,
                **init_params
            )

            # Esegui l'attacco
            y_pred, _, adv_ds, _ = attack.run(samples, labels)

            # Salva i dati dell'attacco
            attack_data[idx] = {
                'x_seq': attack.x_seq,
                'y_pred_adv': y_pred,
                'adv_ds': adv_ds
            }

            print(f"Attacco completato sul modello {idx + 1}")
        except Exception as e:
            print(f"Errore durante l'attacco sul modello {idx + 1}: {e}")
            attack_data[idx] = {'error': str(e)}

    return attack_data


##Saves or loads  attack data on the disk

In [ ]:


# Controlla se il file 'results_FNM.pkl' esiste
if os.path.exists('/content/results_FNM.pkl'):
    print("Il file 'results_FNM' esiste già. Caricamento dei risultati salvati...")
    try:
        with open('/content/results_FNM.pkl', 'rb') as f:
            results_SDF = pickle.load(f)
            if isinstance(results_FNM, list) and all(isinstance(r, dict) for r in results_FNM):
                print("Risultati caricati correttamente.")
            else:
                print("Attenzione: il formato dei dati caricati non è quello previsto.")
    except Exception as e:
        print(f"Errore durante il caricamento: {e}")
else:
    # Se il file non esiste, esegui gli attacchi e salva i risultati
    results_FNM = []
    for idx, model in enumerate(models):
        print(f"Analizzando il modello \"{model_names[idx]}\"...")
        try:
            results= FNM_attack(model, data_tensor, labels_tensor, model_names[idx])
            results_FNM.append(result)
            print(f"Modello: {model_names[idx]} completato.")
        except Exception as e:
            print(f"Errore durante l'analisi del modello {model_names[idx]}: {e}")
            continue

    # Salva i risultati in 'results_SDF.pkl'
    try:
        with open('results_SDF.pkl', 'wb') as f:
            pickle.dump(results_FNM, f)
            print("Risultati salvati nel file 'results_SDF.pkl'.")
    except Exception as e:
        print(f"Errore durante il salvataggio dei risultati: {e}")
